# MySQL connection

Loosely buildt on: https://docs.docker.com/samples/library/mysql/



In [ ]:
%%bash
docker pull mysql:latest

## I want to store the files of the database on the host (my computer, not the container).

I store them in a directory named mysql_databasefiles



In [ ]:
%%bash
# -p flag tells it to "create what is needed" - 
mkdir -p mysql_databasefiles
echo "$(pwd)"

In [ ]:
%%bash
docker run \
--rm \
--name my_mysql \
-v $(pwd)/mysql_databasefiles:/var/lib/mysql \
-p 3306:3306 \
-e MYSQL_ROOT_PASSWORD=deterentysker!42snapsnap \
-d \
mysql
echo "Wooootttt"

### Confirm it runs

In [ ]:
!docker container ls -a

In [ ]:
!ls -l mysql_databasefiles

### Kill it and remove it

In [ ]:
%%bash
docker kill my_mysql
#docker rm my_mysql
docker container ls -a

## Connect to it

In [43]:
import sys
import mysql.connector

def myconnect(user, pw):
        conn = mysql.connector.connect( host='localhost', database='world',user=user, password=pw)
        conn.autocommit = True
        return conn   

rootconn = myconnect('root','deterentysker!42snapsnap')

def sqlQuery(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

def sqlDo(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchwarnings()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

"Done"    

'Done'

In [ ]:
sqlQuery("show databases")

In [ ]:
sqlQuery("show tables")

In [ ]:
sqlQuery("explain world.country")

In [ ]:
sqlQuery("SELECT Name, Code FROM country where Continent = 'Antarctica'")

## Add a new schema

In [ ]:
print ( sqlDo("CREATE DATABASE AWESOMENESS DEFAULT CHARACTER SET 'utf8'"))
"Done"

In [ ]:
print( sqlQuery("show databases"))
"Done"

In [ ]:
print( sqlDo("drop schema if exists AWESOMENESS"))
"Done"

In [ ]:
sqlDo("drop schema if exists AWESOMENESS")

## Explaining a table

In [ ]:
sqlQuery("explain world.country")

In [ ]:
sqlQuery("select Name, GovernmentForm from country where Continent = 'Antarctica'")

# Create a user 

Create a **secret-service** user who
* Can read all world tables
* Alter the GovernmentForm column in the country table


In [23]:
sqlDo("DROP USER IF EXISTS SecretService")

In [24]:
sqlDo("CREATE USER 'SecretService' IDENTIFIED BY 'destable&corrupt'")

In [25]:
sqlQuery("select User, Host from mysql.user")

[('CensusSurveyer', '%'),
 ('SecretService', '%'),
 ('root', '%'),
 ('softadmin', '%'),
 ('mysql.infoschema', 'localhost'),
 ('mysql.session', 'localhost'),
 ('mysql.sys', 'localhost'),
 ('root', 'localhost')]

In [26]:
sqlQuery("explain mysql.user")

[('Host', 'char(60)', 'NO', 'PRI', '', ''),
 ('User', 'char(32)', 'NO', 'PRI', '', ''),
 ('Select_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Insert_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Update_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Delete_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Create_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Drop_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Reload_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Shutdown_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Process_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('File_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Grant_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('References_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Index_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Alter_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Show_db_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Super_priv', "enum('N','Y')", 'NO', '', 'N', ''),
 ('Create_tmp_table_priv', "enum('N','Y')", 'NO'

## Permissions

One can restrict which permissions a user has using this statement:

```sql
GRANT Privilege ON my_database.my_table TO the_user@the_host
```

The following table contains some of the possible values for `Privilege`. The full list is in [the reference manual](https://dev.mysql.com/doc/refman/8.0/en/grant.html).

| Privilege | Meaning and Grantable Levels |
| ----: | :---- | 
| ALL [PRIVILEGES] | Grant all privileges at specified access level except GRANT OPTION and PROXY. |
| ALTER | Enable use of ALTER TABLE. Levels: Global, database, table. |
| CREATE | Enable database and table creation. Levels: Global, database, table. |
| CREATE USER | Enable use of CREATE USER, DROP USER, RENAME USER, and REVOKE ALL PRIVILEGES. Level: Global. |
| DELETE | Enable use of DELETE. Level: Global, database, table. |
| DROP | Enable databases, tables, and views to be dropped. Levels: Global, database, table. |
| GRANT OPTION | Enable privileges to be granted to or removed from other accounts. Levels: Global, database, table, routine, proxy. |
| INSERT | Enable use of INSERT. Levels: Global, database, table, column. |
| SELECT | Enable use of SELECT. Levels: Global, database, table, column. |
| SHOW DATABASES | Enable SHOW DATABASES to show all databases. Level: Global. |
| UPDATE | Enable use of UPDATE. Levels: Global, database, table, column. |
| USAGE | Synonym for “no privileges” |![image.png](attachment:image.png)


In [27]:
sqlQuery("SHOW GRANTS FOR SecretService")

[('GRANT USAGE ON *.* TO `SecretService`@`%`',)]

In [30]:
secretconn = myconnect('SecretService','destable&corrupt')

In [29]:
sqlDo("GRANT SELECT ON world.* TO SecretService")
sqlDo("FLUSH Privileges")

In [32]:
sqlQuery("Select Code, Name, GovernmentForm from country where Continent='Africa' and Population<500000", secretconn)

[('CPV', 'Cape Verde', 'Republic'),
 ('ESH', 'Western Sahara', 'Occupied by Marocco'),
 ('GNQ', 'Equatorial Guinea', 'Republic'),
 ('IOT', 'British Indian Ocean Territory', 'Dependent Territory of the UK'),
 ('MYT', 'Mayotte', 'Territorial Collectivity of France'),
 ('SHN', 'Saint Helena', 'Dependent Territory of the UK'),
 ('STP', 'Sao Tome and Principe', 'Republic'),
 ('SYC', 'Seychelles', 'Republic')]

In [40]:
sqlQuery("Select * from city where CountryCode='ESH'", secretconn)

[(2453, 'El-Aaiún', 'ESH', 'El-Aaiún', 169000),
 (4085, 'DessertHideout', 'ESH', 'Sandworms beware', 1200)]

In [35]:
sqlDo("DELETE FROM city WHERE Name = 'DessertHideout'")

In [39]:
sqlDo("""
INSERT INTO city (Name, CountryCode,District,Population) 
    VALUES ('DessertHideout', 'ESH', 'Sandworms beware', 1200)
""",secretconn)

In [38]:
sqlDo("GRANT UPDATE, INSERT  ON world.city TO SecretService; flush privileges")

## Create one user more

Create censusSurveyer (person who counts inhabitants). 

* This person should be able to read the world database
* **Update the population columns of city and courntry, but nothing else**

In [44]:
sqlDo("DROP USER IF EXISTS CensusSurveyer")

In [45]:
sqlDo("CREATE USER 'CensusSurveyer' IDENTIFIED BY 'count#66behappy'")

In [46]:
sqlDo("GRANT SELECT ON world.* TO CensusSurveyer")
sqlDo("FLUSH Privileges")

In [47]:
sqlQuery("select User, Host from mysql.user")

[('CensusSurveyer', '%'),
 ('SecretService', '%'),
 ('root', '%'),
 ('softadmin', '%'),
 ('mysql.infoschema', 'localhost'),
 ('mysql.session', 'localhost'),
 ('mysql.sys', 'localhost'),
 ('root', 'localhost')]

In [52]:
sqlQuery("SHOW GRANTS FOR CensusSurveyer")

[('GRANT USAGE ON *.* TO `CensusSurveyer`@`%`',),
 ('GRANT SELECT ON `world`.* TO `CensusSurveyer`@`%`',),
 ('GRANT UPDATE (`Population`) ON `world`.`city` TO `CensusSurveyer`@`%`',)]

In [49]:
censusconn = myconnect('CensusSurveyer','count#66behappy')

In [54]:
query = """
SELECT * 
FROM city 
ORDER BY Population DESC
LIMIT 10;
"""
sqlQuery( query,  censusconn)

[(1024, 'Mumbai (Bombay)', 'IND', 'Maharashtra', 10501300),
 (2331, 'Seoul', 'KOR', 'Seoul', 9981619),
 (206, 'São Paulo', 'BRA', 'São Paulo', 9968485),
 (1890, 'Shanghai', 'CHN', 'Shanghai', 9696300),
 (939, 'Jakarta', 'IDN', 'Jakarta Raya', 9604900),
 (2822, 'Karachi', 'PAK', 'Sindh', 9269265),
 (3357, 'Istanbul', 'TUR', 'Istanbul', 8787958),
 (2515, 'Ciudad de México', 'MEX', 'Distrito Federal', 8591309),
 (3580, 'Moscow', 'RUS', 'Moscow (City)', 8389200),
 (3793, 'New York', 'USA', 'New York', 8008278)]

In [51]:
sqlDo("GRANT UPDATE (Population) ON world.city TO CensusSurveyer");
sqlDo("FLUSH Privileges")

In [53]:
sqlDo("UPDATE city SET Population = Population + 1000 WHERE ID=1024",censusconn)

In [ ]:
sqlDo("UPDATE country SET Population = Population + 1000000 WHERE Code='IND'", censusconn)

In [55]:
sqlDo("UPDATE city SET Name = 'Москва' WHERE ID=3580",censusconn)

1143 (42000): UPDATE command denied to user 'CensusSurveyer'@'172.17.0.1' for column 'Name' in table 'city'


### Limiting actions by host

It is possible to grant different permissions depending on from which machine a user is logged in.

There are (at least) three useful scenarios for this:

* Restricting admin users (and in particularly root) to 'localhost'
* Restricting other users to 'localhost', so only users running on the same machine as the DB can access the DB
* Restricting users to a specific set of hosts - so only access from those hosts are valid. Think: DB on one machine and webapps on an other.

## Logs
They serve several roles in serious database maintenance
* Documents all log-ins, and from where
* Documents all queries, for debugging purposes
* Documents performance issues

Besides, the logs play a role in hard recovery of crashed databases

In [ ]:
sqlDo("SET global general_log = 1; SET global log_output = 'table'")

In [59]:
sqlQuery("Explain mysql.general_log")

[('event_time',
  'timestamp(6)',
  'NO',
  '',
  'CURRENT_TIMESTAMP(6)',
  'DEFAULT_GENERATED on update CURRENT_TIMESTAMP(6)'),
 ('user_host', 'mediumtext', 'NO', '', None, ''),
 ('thread_id', 'bigint(21) unsigned', 'NO', '', None, ''),
 ('server_id', 'int(10) unsigned', 'NO', '', None, ''),
 ('command_type', 'varchar(64)', 'NO', '', None, ''),
 ('argument', 'mediumblob', 'NO', '', None, '')]

In [60]:
sqlQuery("""
SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, user_host, command_type, argument 
FROM mysql.general_log
ORDER BY time DESC
LIMIT 10
""")

[('16:06:25', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:25', 'CensusSurveyer[CensusSurveyer] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:25', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:25', 'SecretService[SecretService] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:11', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:11', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('16:01:42',
  'root[root] @  [172.17.0.1]',
  'Query',
  "SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, user_host,  command_type, argument \nFROM mysql.general_log\nWHERE command_type <>'Query'\nORDER BY time DESC\nLIMIT 15"),
 ('16:01:17',
  'root[root] @  [172.17.0.1]',
  'Query',
  "SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, \n    REGEXP_SUBSTR(user_host, '[^ @]+') as user,  command_type, argument \nFROM mysql.general_log\nWHERE command_type <>'Query'\nORDER BY time DESC\nLIMIT 15"),
 ('16:00:40',
  'root[root] @  [172.17.0.1]',
  'Query',
  "SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, \n   

In [61]:
sqlQuery("""
SELECT DATE_FORMAT(event_time,'%H:%i:%s')as time, user_host,  command_type, argument 
FROM mysql.general_log
WHERE command_type <>'Query'
ORDER BY time DESC
LIMIT 15
""")

[('16:06:25', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:25', 'CensusSurveyer[CensusSurveyer] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:25', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:25', 'SecretService[SecretService] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:11', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('16:06:11', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('15:54:42', 'CensusSurveyer[CensusSurveyer] @  [172.17.0.1]', 'Quit', ''),
 ('15:54:42',
  '[CensusSurveyer] @  [172.17.0.1]',
  'Connect',
  'CensusSurveyer@172.17.0.1 on world using SSL/TLS'),
 ('15:23:54', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('15:23:54', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('15:21:15',
  '[root] @  [172.17.0.1]',
  'Connect',
  'root@172.17.0.1 on  using SSL/TLS'),
 ('15:21:15',
  '[root] @  [172.17.0.1]',
  'Connect',
  'root@172.17.0.1 on  using SSL/TLS'),
 ('15:21:08', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('15:21:08', 'root[root] @  [172.17.0.1]', 'Quit', ''),
 ('15

In [62]:
sqlQuery("show tables in mysql")

[('columns_priv',),
 ('component',),
 ('db',),
 ('default_roles',),
 ('engine_cost',),
 ('func',),
 ('general_log',),
 ('global_grants',),
 ('gtid_executed',),
 ('help_category',),
 ('help_keyword',),
 ('help_relation',),
 ('help_topic',),
 ('innodb_index_stats',),
 ('innodb_table_stats',),
 ('password_history',),
 ('plugin',),
 ('procs_priv',),
 ('proxies_priv',),
 ('role_edges',),
 ('server_cost',),
 ('servers',),
 ('slave_master_info',),
 ('slave_relay_log_info',),
 ('slave_worker_info',),
 ('slow_log',),
 ('tables_priv',),
 ('time_zone',),
 ('time_zone_leap_second',),
 ('time_zone_name',),
 ('time_zone_transition',),
 ('time_zone_transition_type',),
 ('user',)]

In [ ]:
sqlQuery("show schemas")

## Backup & restore

The final aspect of security is to be able to recover from a break-down or a hack.

The literature mentions a couple of good concepts which are worth noticing:

* **Physical vs. Logical backup**. Do you backup "all the files of the DB" (Physical), or do you create a (number of) sql files which can be run to restore the database (Logical).
* **Online Versus Offline Backups**. Online backups take place while the MySQL server is running so that the database information can be obtained from the server. Offline backups take place while the server is stopped.
* **Local Versus Remote Backups**. A local backup is performed on the same host where the MySQL server runs, whereas a remote backup is done from a different host.
* **Full Versus Incremental Backups**. A full backup includes all data managed by a MySQL server at a given point in time. An incremental backup consists of the changes made to the data during a given time span.


### full local online logical backup

mysql has a special program named `mysqldump` which can do several of the combinations mentioned above. 

The idea is to backup as: `bash> mysqldump --all-databases > dump.sql`.

Restore can then be done as: `bash> mysql < dump.sql`.



### Storing the backup outside the docker container
I want the backup to end up on my host, not inside the filesystem of the docker container.

We are free to chose where to put it. Inside the container I will store to a directory `/mnt/host`, and map that to `$(pwd)/mysql_dump`. 


In [63]:
%%bash
docker kill my_mysql
docker rm my_mysql
docker container ls -a

my_mysql
my_mysql
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                      NAMES
cc884c1600a3        mongo:latest        "docker-entrypoint.s…"   9 days ago          Up 9 days           0.0.0.0:27017->27017/tcp   mymongo


In [64]:
%%bash
docker run \
--name my_mysql \
-v $(pwd)/mysql_databasefiles:/var/lib/mysql \
-v $(pwd)/mysql_dump:/tmp \
-p 3306:3306 \
-e MYSQL_ROOT_PASSWORD=deterentysker!42snapsnap \
-d \
mysql
echo "Back online"

edd1c8cab28fee3ec2105135379563cab745cfa8d9ff56350db5c2fdd7ccb5fd
Back online


In [65]:
%%bash
docker run \
--name my_mysql2 \
-v $(pwd)/mysql_dump:/tmp \
-e MYSQL_ROOT_PASSWORD=bla_open_now \
-d \
mysql
echo "Back online also"

db597c7c17ba866a46920bb0211748f548100e2e0d4b1537d1097e16671edd57
Back online also
